## 내용

매주 월요일 오전 6시 기준 전주의 데이터를 축적하여 개수의 합을 구하고 summary를 제미나이를 통해서 다시 도출

### 방법
1. sector_detail/"섹터"/dates/"[해당 주차 월요일 일자]"(월-일) 불러오기 
2. summary, counts를 축적시키기
3. 축적된 summary 제미나이에게 보내서 축약하기
4. weeks에 저장


In [1]:
## 시스템 import 위치 지정
import sys
from pathlib import Path

# back 디렉토리를 sys.path에 추가
back_dir = Path(__file__).parent.parent if '__file__' in globals() else Path().resolve().parent
sys.path.append(str(back_dir))

In [2]:
## sectors 불러오기
from data.telegram_raw_datas import sectors

sectors = sectors.keys()

In [3]:
from scripts.firebase.auth import firebase_auth

db = firebase_auth()

In [4]:
from datetime import datetime, timedelta


# 해당일자의 전주 월요일을 받기 위한 함수
## input, output 값: datetime 형식의 날짜
def get_last_monday(date): 
    days_to_this_monday = date.weekday()
    last_monday = date - timedelta(days=days_to_this_monday + 7)
    return last_monday

date = datetime.now()

for i in range(7):
    last_monday = get_last_monday(date)
    current_date = last_monday + timedelta(days=i)
    print(current_date.strftime("%Y-%m-%d"))


2025-08-18
2025-08-19
2025-08-20
2025-08-21
2025-08-22
2025-08-23
2025-08-24


In [21]:
from scripts.gemini.ask_gemini import GeminiClient

gemini_client = GeminiClient()

/Users/apple/Desktop/project/ADA/antopinion/back/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def sum_counts(sector_details):
    total_negative = 0
    total_positive = 0
    total_neutral = 0
    for detail in sector_details:
        if detail and 'counts' in detail:
            counts = detail['counts']
            total_negative += counts.get('negative', 0)
            total_neutral += counts.get('neutral', 0)
            total_positive += counts.get('positive', 0)
    
    return {
        'negative': total_negative,
        'neutral': total_neutral,
        'positive': total_positive
    }

In [23]:
def sum_details(sector_details):
    total_ideas = []
    for detail in sector_details:
        if detail and 'summary' in detail:
            summary = detail['summary']
            total_ideas.append(summary)
    return str(total_ideas)


In [28]:
def get_sector_details(date):
    for sector in sectors:
        sector_details = []
        last_monday = get_last_monday(date)
        for i in range(7):
            current_date = last_monday + timedelta(days=i)
            current_date_str = current_date.strftime("%Y-%m-%d")
            sector_detail = db.collection('sector_detail').document(sector).collection('dates').document(current_date_str).get().to_dict()
            sector_details.append(sector_detail)
        ans = gemini_client.ask_week(sector, sum_details(sector_details))
        print(ans)
        print("--------------------------------")

In [29]:
get_sector_details(date)

{'summary': {'positive': {'headline': '게임스컴 & 한한령 완화 기대감에 게임/엔터 업종 투자 심리 개선', 'summary': '세계 최대 게임 전시회 게임스컴 개최와 중국의 드라마 쿼터 제한 완화 소식에 게임 및 엔터테인먼트 업종에 대한 기대감이 높아지고 있습니다. 펍지 IP의 견조한 성과와 신작 기대감이 투자 심리를 자극하며, 특히 중국 시장 진출 가능성 재부각이 긍정적으로 작용하고 있습니다.'}, 'neutral': {'headline': '게임 산업, 긍정과 부정 요인 혼재…신중한 접근 필요', 'summary': '게임 산업에 대한 다양한 정보와 분석이 공유되고 있지만, 전반적인 투자 심리는 중립적인 경향을 보입니다. 게임 개발 및 관련 산업에 대한 뉴스와 시장 동향이 전달되나, 뚜렷한 매수 또는 매도 의견은 제시되지 않고 있습니다. 투자자들은 성장 가능성과 위험 요인을 신중하게 고려하고 있습니다.'}, 'negative': {'headline': '붉은사막 출시 연기 우려 지속, 펄어비스 투자 심리 악화', 'summary': "펄어비스의 신작 '붉은사막' 출시 연기에 대한 우려가 지속되면서 투자 심리가 악화되고 있습니다. 출시 불확실성 장기화에 따른 투자자 신뢰 하락과 출시 이후 경쟁력에 대한 의문이 제기되며 펄어비스 투자 매력이 감소하고 있다는 평가입니다."}}}
--------------------------------
{'summary': {'positive': {'headline': '조선업, 한미 협력 기대감 속 상승 모멘텀 지속', 'summary': '조선업은 한미 정상회담 및 관련 경제 사절단 구성, 대통령의 조선소 시찰 예정 등 긍정적인 모멘텀이 이어지고 있습니다. 특히 미국 MASGA 프로젝트를 통한 수혜 기대감과 노후 LNG선 교체 수요 증가, LNG 운반선 시장의 호황 등이 투자 심리를 자극하고 있습니다. 다만, 노란봉투법과 같은 국내 규제 환경은 신속한 납기라는 강점을 약화시킬 수 있다는 우려

KeyboardInterrupt: 